# Tutorial по Policyscope

В этом ноутбуке показано, как на синтетических данных сравнить две рекомендательные политики без онлайн A/B‑теста. Мы:
1. сгенерируем пользователей и логи политики A с корректными пропенсити;
2. оценим политику B методами Replay, IPS, SNIPS, DM, Doubly Robust, SNDR и Switch-DR;
3. построим доверительные интервалы через бутстрэп;
4. сравним оффлайн‑оценки с истинным эффектом (оракулом).

## Основные термины

**Пропенсити** (propensity) — вероятность того, что политика A выберет действие `a` в контексте `x`: \(\pi_A(a|x)\). Логи должны содержать эту вероятность для каждого показанного действия.

**Нулевой пропенсити** — ситуация, когда \(\pi_A(a|x)=0\); политика A никогда не выбирает действие `a` при таком `x`. Если политика B хочет выдать это действие, его эффект нельзя оценить на имеющихся данных.

**Стохастическая политика** — политика, которая выбирает действия случайно, согласно распределению вероятностей. Примеры: `epsilon_greedy`, `softmax`. Противоположность — детерминированная политика, всегда отдающая одно действие.

**Value (ценность) политики** — ожидаемое значение целевой метрики при использовании политики. Например, средняя вероятность отклика или средний CLTV при показе действий по этой политике.


## Формат входных логов

Для off-policy оценки нужны логи политики **A** со следующими колонками:
- `user_id` — идентификатор пользователя;
- `a_A` — действие (предложение), показанное A;
- `propensity_A` — вероятность выбрать это действие A;
- `accept` и/или `cltv` — отклик и итоговая ценность;
- признаки пользователя, участвующие в модели.

Важно: `propensity_A` должна быть в диапазоне `(0,1]`; нули делают эффект политики B неидентифицируемым.

## Алгоритмы и их допущения

- **Replay** — использует только совпадения между A и B; требует большого перекрытия действий.
- **IPS** — корректен при известных пропенсити `π_A(a|x)` и `π_B(a|x)`; веса могут иметь большую дисперсию.
- **SNIPS** — нормализует веса IPS, уменьшая дисперсию, но остаётся чувствителен к редким событиям.
- **Direct Method (DM)** — строит модель исхода `μ(x,a)`; смещён, если модель неточна вне области наблюдений.
- **Doubly Robust (DR)** — сочетает DM и IPS; достаточно корректности хотя бы одной из частей.

См. подробности в работах [Dudík et al., 2011](https://arxiv.org/abs/1103.4601) и [Joachims et al., 2017](https://dl.acm.org/doi/10.1145/3018661.3018699).

## 1. Генерация данных

Среда имитирует выдачу банковских предложений. Политика A стохастическая (ε‑greedy), поэтому в логах есть вероятность выбранного действия.

In [1]:
import sys, os
import numpy as np
sys.path.append("../src")

from policyscope.synthetic import SynthConfig, SyntheticRecommenderEnv
from policyscope.policies import make_policy
from policyscope.estimators import (
    train_mu_hat, prepare_piB_taken, replay_value, ips_value, snips_value,
    dm_value, dr_value, sndr_value, switch_dr_value, train_pi_hat, pi_hat_predict
)
from policyscope.bootstrap import paired_bootstrap_ci
import pandas as pd

cfg = SynthConfig(n_users=2000, seed=0)
env = SyntheticRecommenderEnv(cfg)
users = env.sample_users()
users.head()

/workspace/OffPolicyLab/examples/../src/policyscope/estimators.py:127: SyntaxWarning: invalid escape sequence '\h'
  """Обучает модель пропенсити ``\hat\pi(a|x)``.
/workspace/OffPolicyLab/examples/../src/policyscope/estimators.py:144: SyntaxWarning: invalid escape sequence '\h'
  """Предсказывает ``\hat\pi(a|x)`` для всех действий."""
/workspace/OffPolicyLab/examples/../src/policyscope/estimators.py:336: SyntaxWarning: invalid escape sequence '\h'
  Обученная модель `\hat\mu(x,a)`.
/workspace/OffPolicyLab/examples/../src/policyscope/estimators.py:388: SyntaxWarning: invalid escape sequence '\('
  \(\frac{1}{n} \sum_i \big[q^\,(x_i, \pi_B) + \frac{w_i}{\bar{w}} (r_i - q^\,(x_i, a_i))\big]\).
/workspace/OffPolicyLab/examples/../src/policyscope/estimators.py:452: SyntaxWarning: invalid escape sequence '\('
  Добавляет IPS-поправку только если вес \(w_i = \pi_B/\pi_A\) не превышает


,user_id,loyal,age,risk,income,region
0,0,0,39,0.164427,33847.243699,2
1,1,1,68,0.794970,28464.133287,1
2,2,1,51,0.173588,106405.425932,1
3,3,1,21,0.460746,23807.909876,0
4,4,0,58,0.129342,22354.664095,2


## 2. Политики A и B

- **A**: ε‑greedy (ε=0.15) на скрытых скоринговых функциях.
- **B**: softmax (τ=0.7) на тех же скорингах.

Сгенерируем логи A и посмотрим первые строки.

In [2]:

policyA = make_policy('epsilon_greedy', seed=1, epsilon=0.15)
policyB = make_policy('softmax', seed=2, tau=0.7)
logsA = env.simulate_logs_A(policyA, users)
logsA.head()


,user_id,loyal,age,risk,income,region,a_A,accept,cltv
0,0,0,39,0.164427,33847.243699,2,0,1,535.999459
1,1,1,68,0.794970,28464.133287,1,1,0,606.417789
2,2,1,51,0.173588,106405.425932,1,0,1,1555.978860
3,3,1,21,0.460746,23807.909876,0,2,1,752.871677
4,4,0,58,0.129342,22354.664095,2,0,1,472.191265


## 3. Оценка value(A) и value(B)

Метрики:
- **accept** — отклик пользователя (0/1);
- **cltv** — изменённая ценность за горизонт 90 дней.

Рассчитаем V(A) и оценки V(B) разными методами.

_Функции `train_pi_hat` и `train_mu_hat` автоматически нормализуют поля `age`, `risk` и `income`, поэтому подготовка данных не требует ручного масштабирования._

In [3]:
vA_accept = logsA['accept'].mean()
vA_cltv = logsA['cltv'].mean()

pi_model = train_pi_hat(logsA)
pA_all = pi_hat_predict(pi_model, logsA)
pA_taken = pA_all[np.arange(len(logsA)), logsA['a_A'].values]

piB_taken = prepare_piB_taken(logsA, policyB)
mu_accept = train_mu_hat(logsA, target='accept')
mu_cltv = train_mu_hat(logsA, target='cltv')

res = []
# Replay
res.append(('Replay',
             replay_value(logsA, policyB.action_argmax(users), 'accept'),
             replay_value(logsA, policyB.action_argmax(users), 'cltv')))
# IPS
res_ip_accept, ess_ip_a, clip_ip_a = ips_value(logsA, piB_taken, pA_taken, 'accept', weight_clip=20)
res_ip_cltv, ess_ip_c, clip_ip_c = ips_value(logsA, piB_taken, pA_taken, 'cltv', weight_clip=20)
res.append(('IPS', res_ip_accept, res_ip_cltv))
# SNIPS
res_sn_accept, ess_sn_a, clip_sn_a = snips_value(logsA, piB_taken, pA_taken, 'accept', weight_clip=20)
res_sn_cltv, ess_sn_c, clip_sn_c = snips_value(logsA, piB_taken, pA_taken, 'cltv', weight_clip=20)
res.append(('SNIPS', res_sn_accept, res_sn_cltv))
# DM
res.append(('DM', dm_value(logsA, policyB, mu_accept, 'accept'), dm_value(logsA, policyB, mu_cltv, 'cltv')))
# SNDR
res_sndr_accept, ess_sndr_a, clip_sndr_a = sndr_value(logsA, policyB, mu_accept, pA_taken, 'accept', weight_clip=20)
res_sndr_cltv, ess_sndr_c, clip_sndr_c = sndr_value(logsA, policyB, mu_cltv, pA_taken, 'cltv', weight_clip=20)
res.append(('SNDR', res_sndr_accept, res_sndr_cltv))
# Switch-DR
res_sw_accept, ess_sw_a, share_sw_a = switch_dr_value(logsA, policyB, mu_accept, pA_taken, tau=20, target='accept')
res_sw_cltv, ess_sw_c, share_sw_c = switch_dr_value(logsA, policyB, mu_cltv, pA_taken, tau=20, target='cltv')
res.append(('Switch-DR', res_sw_accept, res_sw_cltv))
# DR
res_dr_accept, ess_dr_a, clip_dr_a = dr_value(logsA, policyB, mu_accept, pA_taken, 'accept', weight_clip=20)
res_dr_cltv, ess_dr_c, clip_dr_c = dr_value(logsA, policyB, mu_cltv, pA_taken, 'cltv', weight_clip=20)
res.append(('DR', res_dr_accept, res_dr_cltv))

pd.DataFrame(res, columns=['method','V_B_accept','V_B_cltv'])

/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


,method,V_B_accept,V_B_cltv
0,Replay,0.528015,720.610675
1,IPS,0.522772,693.919663
2,SNIPS,0.532812,707.246642
3,DM,0.492770,681.656283
4,SNDR,0.508577,676.029570
5,Switch-DR,0.493631,680.986434
6,DR,0.508279,676.135597


## 4. Доверительные интервалы для DR

Используем кластерный бутстрэп по пользователям, чтобы получить 95% доверительные интервалы для разности ценностей (ATE).

In [4]:
def estimator_pair_accept(df):
    mu = train_mu_hat(df, 'accept')
    pi_model = train_pi_hat(df)
    pA = pi_hat_predict(pi_model, df)
    pA_taken = pA[np.arange(len(df)), df['a_A'].values]
    vA = df['accept'].mean()
    vB, _, _ = dr_value(df, policyB, mu, pA_taken, 'accept', weight_clip=20)
    return vA, vB, vB - vA


def estimator_pair_cltv(df):
    mu = train_mu_hat(df, 'cltv')
    pi_model = train_pi_hat(df)
    pA = pi_hat_predict(pi_model, df)
    pA_taken = pA[np.arange(len(df)), df['a_A'].values]
    vA = df['cltv'].mean()
    vB, _, _ = dr_value(df, policyB, mu, pA_taken, 'cltv', weight_clip=20)
    return vA, vB, vB - vA

ci_accept = paired_bootstrap_ci(logsA, estimator_pair_accept)
ci_cltv = paired_bootstrap_ci(logsA, estimator_pair_cltv)
ci_accept, ci_cltv

/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logisti

/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logisti

/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logisti

/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logisti

/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logisti

...

/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logisti

/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logisti

/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/root/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


({'V_A': np.float64(0.514),
  'V_A_CI': (0.49940092222639487, 0.5294511917176095),
  'V_B': 0.5082788300703585,
  'V_B_CI': (0.4763451170613972, 0.5378790011611967),
  'Delta': np.float64(-0.005721169929641556),
  'Delta_CI': (-0.03349549791901197, 0.018270048271229092),
  'n_boot': 300},
 {'V_A': np.float64(689.4446007608683),
  'V_A_CI': (676.3891830223065, 700.2707381250596),
  'V_B': 676.1355967680565,
  'V_B_CI': (660.9344367229127, 690.3560962733751),
  'Delta': np.float64(-13.309003992811768),
  'Delta_CI': (-24.262302226522095, -3.2479350515259613),
  'n_boot': 300})

## 5. Сравнение с "Оракулом"

Так как мы знаем генеративную модель, можно вычислить истинные ожидания V(A) и V(B) и проверить точность OPE.

In [5]:
oracle_A_accept = env.oracle_value(policyA, users, metric="accept")
oracle_B_accept = env.oracle_value(policyB, users, metric="accept")
oracle_A_cltv = env.oracle_value(policyA, users, metric="cltv")
oracle_B_cltv = env.oracle_value(policyB, users, metric="cltv")

# DR estimates
DR_V_B_accept = res_dr_accept
DR_V_B_cltv = res_dr_cltv
DR_Delta_accept = res_dr_accept - vA_accept
DR_Delta_cltv = res_dr_cltv - vA_cltv

oracle_df = pd.DataFrame({
    "metric": ["accept", "cltv"],
    "V_A_true": [oracle_A_accept, oracle_A_cltv],
    "V_B_true": [oracle_B_accept, oracle_B_cltv],
    "Delta_true": [oracle_B_accept - oracle_A_accept, oracle_B_cltv - oracle_A_cltv],
    "DR_V_B": [DR_V_B_accept, DR_V_B_cltv],
    "DR_Delta": [DR_Delta_accept, DR_Delta_cltv],
    "abs_error_V_B": [abs(DR_V_B_accept - oracle_B_accept), abs(DR_V_B_cltv - oracle_B_cltv)],
    "abs_error_Delta": [abs(DR_Delta_accept - (oracle_B_accept - oracle_A_accept)),
                         abs(DR_Delta_cltv - (oracle_B_cltv - oracle_A_cltv))]
})
oracle_df

,metric,V_A_true,V_B_true,Delta_true,DR_V_B,DR_Delta,abs_error_V_B,abs_error_Delta
0,accept,0.541000,0.516000,-0.025000,0.508279,-0.005721,0.007721,0.019279
1,cltv,698.152897,682.062847,-16.090049,676.135597,-13.309004,5.927250,2.781045


### Таблица сравнения
Столбцы:
- **V_A_true**, **V_B_true** — истинные средние значения метрики под политиками A и B.
- **Delta_true** — истинная разница V_B_true - V_A_true.
- **DR_V_B** — оценка V(B) методом Doubly Robust.
- **DR_Delta** — оценка разницы V(B)-V(A) методом DR.
- **abs_error_V_B**, **abs_error_Delta** — абсолютные ошибки оценок относительно истины.


## 6. Вывод

- В синтетике политика **B** уступает A: DR-оценка разницы совпадает по знаку с истинной дельтой.
- По отклику и CLTV абсолютная ошибка DR-аппроксимации невелика (см. таблицу выше).
- Это демонстрирует, что при корректных пропенсити и перекрытии действий метод DR может заменить online A/B для предварительных оценок.
